# Experiment

> Experiments hold the results of an experiment against a dataset.

In [1]:
# | default_exp experiment

In [2]:
# | export
import typing as t

from fastcore.utils import patch

from ragas_experimental.model.pydantic_model import ExtendedPydanticBaseModel as BaseModel
from ragas_experimental.backends.ragas_api_client import RagasApiClient
from ragas_experimental.dataset import Dataset

/Users/jjmachan/workspace/eglabs/ragas/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# | export
class Experiment(Dataset):
    def __init__(
        self,
        name: str,
        model: t.Type[BaseModel],
        project_id: str,
        experiment_id: str,
        ragas_api_client: t.Optional[RagasApiClient] = None,
        backend: t.Literal["ragas_app", "local"] = "ragas_app",
        local_root_dir: t.Optional[str] = None,
    ):
        self.experiment_id = experiment_id
        super().__init__(
            name=name, 
            model=model, 
            project_id=project_id, 
            dataset_id=experiment_id, 
            ragas_api_client=ragas_api_client,
            backend=backend,
            local_root_dir=local_root_dir,
            datatable_type="experiments",
        )

    def __str__(self):
        return f"Experiment(name={self.name}, model={self.model.__name__}, len={len(self._entries)})"

    __repr__ = __str__

In [4]:
from ragas_experimental import Project, BaseModel
from ragas_experimental.metric import MetricResult
from ragas_experimental.utils import get_test_directory
import typing as t

In [5]:
root_dir = get_test_directory()

p = Project(
    project_id="test",
    backend="local",
    root_dir=root_dir,
)

class TestDataRow(BaseModel):
    id: t.Optional[int]
    query: str
    persona: t.List[t.Literal["opt1", "opt2", "opt3"]]

dataset = p.create_dataset(
    name="test_dataset",
    model=TestDataRow,
)

for i in range(3):
    row = TestDataRow(id=i, query=f"query_{i}", persona=["opt1"])
    dataset.append(row)


In [6]:
i = 0
row = TestDataRow(id=i, query=f"query_{i}", persona=["opt1"])

In [7]:
fields_info = {
    field_name: {
        'value': getattr(row, field_name),
        'type': field_info.annotation
    }
    for field_name, field_info in row.model_fields.items()
}

fields_info

{'id': {'value': 0, 'type': typing.Optional[int]},
 'query': {'value': 'query_0', 'type': str},
 'persona': {'value': ['opt1'],
  'type': typing.List[typing.Literal['opt1', 'opt2', 'opt3']]}}

In [9]:
class ExperimentDataRow(TestDataRow):
    response: str 
    metrics: t.List[MetricResult]

In [17]:
for d in dataset:
    response = "test"
    score1 = MetricResult(result=1, reason="score 1")
    score2 = MetricResult(result=0, reason="score 2")

    e = ExperimentDataRow(
        id=row.id,
        query=row.query,
        persona=["opt1"],
        response=response,
        metrics=[score1, score2],
    )
    experiments.append(e)

In [19]:
import os

os.listdir(
    os.path.join(root_dir, "test", "experiments")
)

['test_experiment.csv']